In [1]:
# IMPORT 

import csv
from biomart import BiomartServer
import pandas as pd
import openpyxl

In [2]:
# DATASET

## biomart_server
server = BiomartServer('http://www.ensembl.org/biomart')
dataset = server.datasets['rnorvegicus_gene_ensembl']

In [3]:
# FUNCTIONS

def gene_dictionary(index,
                    gene_name,
                    gene_list_number,
                    gene_list_id,
                    source,
                    ensembl_gene_id,
                    ensembl_transcript_id,
                    refseq_mrna,
                    hgnc_symbol,
                    organism,
                    alias,
                    direction_string):
    
    dictionary = {
        'Index' : index,
        'Gene_name' : gene_name,
        'Gene_list_number' : gene_list_number,
        'Gene_list_index' : gene_list_id,
        'Source' : source,
        'Ensembl_gene_id' : ensembl_gene_id,
        'Ensembl_transcript_id' : ensembl_transcript_id,
        'RefSeq_mRNA_id' : refseq_mrna,
        'Organism' : organism,
        'HGNC_symbol' : hgnc_symbol,
        'Alias' : alias,
        'Info' : direction_string
    }
    
    return dictionary


  
def biomartParameters(mgi_symbol):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']

    filters = {'external_gene_name':[mgi_symbol]}               # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values  


def biomartHumanOrthologs(mgi_symbol):
    
    attributes = ['hsapiens_homolog_associated_gene_name']
    filters = {'external_gene_name':[mgi_symbol]}
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
    
    return values  
    
def biomartParametersbyEnsembl(ensembl_id):

    attributes = ['external_gene_name',
                  'ensembl_transcript_id',
                  'refseq_mrna']

    filters = {'ensembl_gene_id':[ensembl_gene_id]}               # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values  

def biomartHumanOrthologsbyEnsembl(ensembl_id):

    attributes = ['hsapiens_homolog_associated_gene_name']

    filters = {'ensembl_gene_id':[ensembl_gene_id]}               # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values  




def alias_and_official(ls_notResponse,ls_row_10,ls_row_2):

    ls_response = []
    ls_response2 = []
    
    for notResponse in ls_notResponse:
        word = str(notResponse).lower()

        for i in range(len(ls_row_10)):
            if ls_row_10[i]:
                temp = str(ls_row_10[i]).lower()
                t_strings = temp.split("|")
                if word in t_strings:
                    ls_response.append([word, i, ls_row_1[i]])

    ls_response.insert(0, ['gene_name','alias_index','mgi_id'])         # print(len(ls_response))


    for notResponse in ls_notResponse:
        word = str(notResponse).lower()

        for i in range(len(ls_row_10)):
            if ls_row_10[i]:
                temp = str(ls_row_10[i]).lower()
                if word == temp:
                    ls_response2.append([word, i, ls_row_1[i]])
                    

    ls_response.insert(0, ['gene_name','official_index','mgi_id'])      # print(len(ls_response2))

    ls_response_3 = ls_response
    
    for response in ls_response2:
        if response not in ls_response_3:
            ls_response_3.append(response)
        else: print(response)

    ls_response_3.pop(0)
    ls_response_3.pop(0)
    ls_response.insert(0, ['gene_name','official/alias_index','mgi_id'])
    
    return ls_response_3

def biomartParameters_mgi(mgi_id):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']
    filters = {'mgi_id':[mgi_id]}                     # gene_name = mgi_id
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
    
    return values  


def biomartHumanOrthologs_mgi(mgi_id):
    attributes = ['hsapiens_homolog_associated_gene_name']
    
    filters = {'mgi_id':[mgi_id]}
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
    
    return values  


def updateCellswithAlias(mgi_file_path, dictionary_file_path, file_path):
    
    # Load MGI file and open workbook
    wb_mgi = openpyxl.load_workbook(mgi_file_path)
    ws_mgi = wb_mgi.active

    ls_row_alias = [ws_mgi.cell(row=i,column=10).value for i in range(2,ws_mgi.max_row+1)]
    ls_row_g_name = [ws_mgi.cell(row=i,column=2).value for i in range(2,ws_mgi.max_row+1)] 

    # Load dictionary file and open workbook
    workbook = openpyxl.load_workbook(dictionary_file_path)
    sheet = workbook.active

    ls_row_2 = [sheet.cell(row=i,column=2).value for i in range(2,sheet.max_row+1)]
    ls_row_9 = [sheet.cell(row=i,column=11).value for i in range(2,sheet.max_row+1)]

    
    length = len(ls_row_2) + 2

    # Modify the desired cell
    for i in range(length):
        K_cel = 'K' + str(i + 2)

        for k in range(len(ls_row_g_name)):
            try:
                if str(ls_row_g_name[k]).lower() == str(ls_row_2[i]).lower():
                    if ls_row_alias[k] is not None:
                        sheet[K_cel] = ls_row_alias[k]
                    else:
                        continue
            except:
                continue

    # Save the file
    workbook.save(file_path)


In [4]:
# LOAD START FILES

## PUBLICATION SOURCE
wb = openpyxl.load_workbook('../ifpan-GR-database-papers.xlsx')
ws = wb['hipocamp-fs']

publication_geneName = [ws.cell(row=i,column=3).value for i in range(2,ws.max_row+1)]
direction = [ws.cell(row=i,column=6).value for i in range(2,ws.max_row+1)]
Ensembl_id = [ws.cell(row=i,column=4).value for i in range(2,ws.max_row+1)]


## MGI
wb_1 = openpyxl.load_workbook(r'../MGI_EntrezGene.xlsx')
ws_1 = wb_1.active

ls_row_10 = [ws_1.cell(row=i,column=10).value for i in range(2,ws_1.max_row+1)]
ls_row_2 = [ws_1.cell(row=i,column=2).value for i in range(2,ws_1.max_row+1)]
ls_row_1 = [ws_1.cell(row=i,column=1).value for i in range(2,ws_1.max_row+1)]  

In [5]:
print(Ensembl_id)

['ENSRNOG00000007830', 'ENSRNOG00000003977', 'ENSRNOG00000014350', 'ENSRNOG00000003300', 'ENSRNOG00000008015', 'ENSRNOG00000007607', 'ENSRNOG00000011815', 'ENSRNOG00000007390', 'ENSRNOG00000042838', 'ENSRNOG00000058383', 'ENSRNOG00000006972', 'ENSRNOG00000054344', 'ENSRNOG00000003745', 'ENSRNOG00000001189', 'ENSRNOG00000019422', 'ENSRNOG00000058388', 'ENSRNOG00000061081', 'ENSRNOG00000043465', 'ENSRNOG00000029336', 'ENSRNOG00000014205', 'ENSRNOG00000045649', 'ENSRNOG00000021027', 'ENSRNOG00000046667', 'ENSRNOG00000007387', 'ENSRNOG00000005387', 'ENSRNOG00000047977', 'ENSRNOG00000017412', 'ENSRNOG00000051552', 'ENSRNOG00000051570', 'ENSRNOG00000058727', 'ENSRNOG00000057008', 'ENSRNOG00000011263', 'ENSRNOG00000014011', 'ENSRNOG00000054978', 'ENSRNOG00000022839', 'ENSRNOG00000016299', 'ENSRNOG00000045554', 'ENSRNOG00000058186', 'ENSRNOG00000023465', 'ENSRNOG00000007152', 'ENSRNOG00000057126', 'ENSRNOG00000052226', 'ENSRNOG00000023896', 'ENSRNOG00000015036', 'ENSRNOG00000025476', 'ENSRNOG0

In [6]:
# LISTS

dictionary = {}
ls_geneDictionaries = []
ls_notResponse = []
ls_notResponse_after = []


In [7]:
# DICTIONARY

for i in range(len(publication_geneName)):
    gene_name = publication_geneName[i]
    direction_string = direction[i]   # => info
    
    # VARIABLES
    index = i + 1
    gene_list_number = 31
    gene_list_id =  'all_sig_genes_34362910'      # => cluster
    source = 'PMID: 34362910'
    organism = 'rat'
    print(direction_string)
    alias = ''
    
    if gene_name != '':
        
        if gene_name == 'NA':
            
            ensembl_gene_id = Ensembl_id[i]
            external_gene_name_temp = []
            ensembl_transcript_id_temp = []
            refseq_mrna_temp = []

            ls_biomartParametersbyEnsembl = biomartParametersbyEnsembl(ensembl_gene_id)
            if not ls_biomartParametersbyEnsembl:
                print('no ensembl')


            for ls in ls_biomartParametersbyEnsembl:
                for j in range(3):
                    if len(ls) < (j+1):
                        ls.append('')
                external_gene_name_temp.append(ls[0])
                ensembl_transcript_id_temp.append(ls[1])
                refseq_mrna_temp.append(ls[2])  

            gene_name = '|'.join(list(set(filter(None, external_gene_name_temp))))
            ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
            refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))
            
            print(gene_name)
            
            # ORTHOLOGS
            ls_biomartHumanOrthologsbyEnsembl = biomartHumanOrthologsbyEnsembl(ensembl_gene_id)
            if not ls_biomartHumanOrthologsbyEnsembl:
                hgnc_symbol = ''
            else:
                hgnc_symbol = ls_biomartHumanOrthologsbyEnsembl[0][0]
            
            
            
        else:

            ensembl_gene_id_temp = []
            ensembl_transcript_id_temp = []
            refseq_mrna_temp = []


            ls_biomartParameters = biomartParameters(gene_name)
            if not ls_biomartParameters:
                ls_notResponse.append(gene_name)
                print(ls_notResponse)


            for ls in ls_biomartParameters:
                for j in range(3):
                    if len(ls) < (j+1):
                        ls.append('')
                ensembl_gene_id_temp.append(ls[0])
                ensembl_transcript_id_temp.append(ls[1])
                refseq_mrna_temp.append(ls[2])  

            ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
            ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
            refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))


            # ORTHOLOGS
            ls_biomartHumanOrthologs = biomartHumanOrthologs(gene_name)
            if not ls_biomartHumanOrthologs:
                hgnc_symbol = ''
            else:
                hgnc_symbol = ls_biomartHumanOrthologs[0][0]
        
        temp_gene_dictionary = gene_dictionary(index,
                                               gene_name,
                                               gene_list_number,
                                               gene_list_id,
                                               source,
                                               ensembl_gene_id,
                                               ensembl_transcript_id, 
                                               refseq_mrna,
                                               hgnc_symbol,
                                               organism,
                                               alias,
                                               direction_string)

        ls_geneDictionaries.append(temp_gene_dictionary)
        print(temp_gene_dictionary)





up
{'Index': 1, 'Gene_name': 'Apold1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000007830', 'Ensembl_transcript_id': 'ENSRNOT00000102557|ENSRNOT00000010289', 'RefSeq_mRNA_id': 'NM_001003403', 'Organism': 'rat', 'HGNC_symbol': 'APOLD1', 'Alias': '', 'Info': 'up'}
up
['Dusp1']
{'Index': 2, 'Gene_name': 'Dusp1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
up
['Dusp1', 'Cyr61']
{'Index': 3, 'Gene_name': 'Cyr61', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
up
{'Index': 4, 'Gene_name': 'Btg2', 'Gene_list_number': 31, 'Gene_list_ind

['Dusp1', 'Cyr61', 'Rbm3', 'LOC100910163']
{'Index': 26, 'Gene_name': 'LOC100910163', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
up
{'Index': 27, 'Gene_name': 'AABR07001512.1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000017412', 'Ensembl_transcript_id': 'ENSRNOT00000023387', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
up
no ensembl

{'Index': 28, 'Gene_name': '', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000051552', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
up
{'Index': 29, 'Gene_name': 'AABR07001910.1', 'Gene_list

{'Index': 51, 'Gene_name': 'Oca2', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000014465', 'Ensembl_transcript_id': 'ENSRNOT00000108070|ENSRNOT00000019498', 'RefSeq_mRNA_id': 'NM_001271493', 'Organism': 'rat', 'HGNC_symbol': 'OCA2', 'Alias': '', 'Info': 'up'}
down
['Dusp1', 'Cyr61', 'Rbm3', 'LOC100910163', 'Hist1h1c', 'LOC102551901', 'LOC500300', 'Ctgf', 'Cml3', 'RGD1560017']
{'Index': 52, 'Gene_name': 'RGD1560017', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'down'}
up
{'Index': 53, 'Gene_name': 'Tmem119', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000000700', 'Ensembl_transcript_id': 'ENSRNOT00000000889|ENSRNOT00000099644', 'RefSeq_mRNA_id

{'Index': 74, 'Gene_name': 'Junb', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000067725', 'Ensembl_transcript_id': 'ENSRNOT00000108644', 'RefSeq_mRNA_id': 'NM_021836', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
up
no ensembl

{'Index': 75, 'Gene_name': '', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000058727', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
up
{'Index': 76, 'Gene_name': 'Sik1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000001189', 'Ensembl_transcript_id': 'ENSRNOT00000103104|ENSRNOT00000001579', 'RefSeq_mRNA_id': 'NM_021693', 'Organism': 'rat', 'HGNC_symbol': 'SIK1', 'Alias': '', 'Info': 'up'}
up
{'Index': 77, 'Gene_name': 'Gpd1', 'Gen

{'Index': 98, 'Gene_name': 'Lfng', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000001250', 'Ensembl_transcript_id': 'ENSRNOT00000001682|ENSRNOT00000101623', 'RefSeq_mRNA_id': 'NM_133393', 'Organism': 'rat', 'HGNC_symbol': 'LFNG', 'Alias': '', 'Info': 'up'}
up
{'Index': 99, 'Gene_name': 'Arrdc2', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000019009', 'Ensembl_transcript_id': 'ENSRNOT00000025956', 'RefSeq_mRNA_id': 'NM_001107303', 'Organism': 'rat', 'HGNC_symbol': 'ARRDC2', 'Alias': '', 'Info': 'up'}
down
{'Index': 100, 'Gene_name': 'Tnfrsf11b', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000008336', 'Ensembl_transcript_id': 'ENSRNOT00000011344|ENSRNOT00000096742|ENSRNOT00000102998', 'RefSeq_mRNA_id': 'NM_012870', 'Organism': 'rat', 'HGNC_symbol

...

no ensembl

{'Index': 4454, 'Gene_name': '', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000053803', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
down
{'Index': 4455, 'Gene_name': 'Frmpd1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000012546', 'Ensembl_transcript_id': 'ENSRNOT00000016839', 'RefSeq_mRNA_id': 'NM_001107937', 'Organism': 'rat', 'HGNC_symbol': 'FRMPD1', 'Alias': '', 'Info': 'down'}
up
['Dusp1', 'Cyr61', 'Rbm3', 'LOC100910163', 'Hist1h1c', 'LOC102551901', 'LOC500300', 'Ctgf', 'Cml3', 'RGD1560017', 'AABR07002120.1', 'LOC684871', 'LOC100912293', 'Dusp1', 'LOC100909555', 'LOC100363268', 'LOC500300', 'LOC102557302', 'LOC500594', 'LOC297756', 'Ctgf', 'RGD1561282', 'LOC100911993', 'Clec2j', 'Lyst', 'Igkc', 'LOC100360573', 'Rbm3', 'RGD1306926', 'Cc

['Dusp1', 'Cyr61', 'Rbm3', 'LOC100910163', 'Hist1h1c', 'LOC102551901', 'LOC500300', 'Ctgf', 'Cml3', 'RGD1560017', 'AABR07002120.1', 'LOC684871', 'LOC100912293', 'Dusp1', 'LOC100909555', 'LOC100363268', 'LOC500300', 'LOC102557302', 'LOC500594', 'LOC297756', 'Ctgf', 'RGD1561282', 'LOC100911993', 'Clec2j', 'Lyst', 'Igkc', 'LOC100360573', 'Rbm3', 'RGD1306926', 'Ccrn4l', 'RGD1564688', 'LOC499229', 'AABR07046778.1', 'LOC102552378', 'Paqr9', 'Mkl1', 'Elmsan1', 'Hist1h1d', 'AABR07015081.2', 'AABR07059879.1', 'LOC100363268', 'Lyst', 'LOC100360977', 'Rbm3', 'AABR07061336.3', 'Pak7', 'AABR07028989.1', 'LOC100366140', 'Ogn', 'LOC100910934', 'Cml5', 'Rbm3', 'LOC100360573', 'Tmem151b', 'Cecr6', 'Cml5', 'Dusp1', 'AABR07026359.1', 'Pqlc1', 'Lyst', 'Sdpr', 'Gramd3', 'Ddit4l2', 'Bcl7a', 'LOC100364791', 'Sh3bp5', 'Gucy1b3', 'Slc25a3', 'Rhoq', 'AABR07001905.1', 'RGD1562037', 'LOC100911874', 'LOC100912399', 'Cyr61', 'Pappa', 'Rn60_1_2787.1', 'H3f3b', 'LOC304239', 'Pragmin', 'LOC100912399', 'LOC100360638', 

{'Index': 4472, 'Gene_name': 'Slc17a7', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000020650', 'Ensembl_transcript_id': 'ENSRNOT00000028064|ENSRNOT00000064184', 'RefSeq_mRNA_id': 'NM_053859', 'Organism': 'rat', 'HGNC_symbol': 'SLC17A7', 'Alias': '', 'Info': 'up'}
up
{'Index': 4473, 'Gene_name': 'Cables1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000012795', 'Ensembl_transcript_id': 'ENSRNOT00000104819|ENSRNOT00000017187', 'RefSeq_mRNA_id': 'NM_001107404', 'Organism': 'rat', 'HGNC_symbol': 'CABLES1', 'Alias': '', 'Info': 'up'}
down
{'Index': 4474, 'Gene_name': 'Runx1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000001704', 'Ensembl_transcript_id': 'ENSRNOT00000002313|ENSRNOT00000084134|ENSRNOT00000096598', 'RefSeq_mRNA_id': 'NM_017325', 'Or

{'Index': 4488, 'Gene_name': 'AABR07015057.1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'down'}
up
{'Index': 4489, 'Gene_name': 'Daxx', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000000477', 'Ensembl_transcript_id': 'ENSRNOT00000000559|ENSRNOT00000119385|ENSRNOT00000092597', 'RefSeq_mRNA_id': 'NM_080891', 'Organism': 'rat', 'HGNC_symbol': 'DAXX', 'Alias': '', 'Info': 'up'}
up
{'Index': 4490, 'Gene_name': 'Pld5', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': 'ENSRNOG00000003997', 'Ensembl_transcript_id': 'ENSRNOT00000005332|ENSRNOT00000078195', 'RefSeq_mRNA_id': 'NM_001191674', 'Organism': 'rat', 'HGNC_symbol': 'PLD5', 'Alias': '', 'Info': 'up'}
down


{'Index': 4511, 'Gene_name': 'Pqlc1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_34362910', 'Source': 'PMID: 34362910', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}


In [8]:
print(ls_notResponse)

['Dusp1', 'Cyr61', 'Rbm3', 'LOC100910163', 'Hist1h1c', 'LOC102551901', 'LOC500300', 'Ctgf', 'Cml3', 'RGD1560017', 'AABR07002120.1', 'LOC684871', 'LOC100912293', 'Dusp1', 'LOC100909555', 'LOC100363268', 'LOC500300', 'LOC102557302', 'LOC500594', 'LOC297756', 'Ctgf', 'RGD1561282', 'LOC100911993', 'Clec2j', 'Lyst', 'Igkc', 'LOC100360573', 'Rbm3', 'RGD1306926', 'Ccrn4l', 'RGD1564688', 'LOC499229', 'AABR07046778.1', 'LOC102552378', 'Paqr9', 'Mkl1', 'Elmsan1', 'Hist1h1d', 'AABR07015081.2', 'AABR07059879.1', 'LOC100363268', 'Lyst', 'LOC100360977', 'Rbm3', 'AABR07061336.3', 'Pak7', 'AABR07028989.1', 'LOC100366140', 'Ogn', 'LOC100910934', 'Cml5', 'Rbm3', 'LOC100360573', 'Tmem151b', 'Cecr6', 'Cml5', 'Dusp1', 'AABR07026359.1', 'Pqlc1', 'Lyst', 'Sdpr', 'Gramd3', 'Ddit4l2', 'Bcl7a', 'LOC100364791', 'Sh3bp5', 'Gucy1b3', 'Slc25a3', 'Rhoq', 'AABR07001905.1', 'RGD1562037', 'LOC100911874', 'LOC100912399', 'Cyr61', 'Pappa', 'Rn60_1_2787.1', 'H3f3b', 'LOC304239', 'Pragmin', 'LOC100912399', 'LOC100360638', 

In [9]:
# SCORES 

### from DICTIONARY to file
data = ls_geneDictionaries
df = pd.DataFrame.from_dict(data)
df.to_excel(r'.\Dictionary.xlsx', index=False)

### from ls_notResponse to file
data1 = ls_notResponse
df1 = pd.DataFrame.from_dict(data1)
df1.to_excel(r'.\notResponse.xlsx', index=False)

# LOAD 
# notRESPONSE
wb_2 = openpyxl.load_workbook(r'.\notResponse.xlsx')
ws_2 = wb_2.active
ls_notResponse_v2 = [ws_2.cell(row=i,column=1).value for i in range(2,ws_2.max_row+1)]


### from alias_and_official to file
data2 = alias_and_official(ls_notResponse, ls_row_10, ls_row_2)
df2 = pd.DataFrame.from_dict(data2)
df2.to_excel(r'.\responsewithMGI.xlsx', index=False)


# LOAD responsewithMGI
wb_3 = openpyxl.load_workbook(r'.\responsewithMGI.xlsx')
ws_3 = wb_3.active
mgi_id = [ws_3.cell(row=i,column=3).value for i in range(3,ws_3.max_row+1)]
gene_name_v2 = [ws_3.cell(row=i,column=1).value for i in range(3,ws_3.max_row+1)]

['cecr6', 66485, 'MGI:2136977']
['wdr92', 68872, 'MGI:2144224']
['lhfp', 79733, 'MGI:1891214']
['4930452b06rik', 64726, 'MGI:1926154']
['sepw1', 30705, 'MGI:1100878']
['agpat6', 68374, 'MGI:2142716']
['1700007b14rik', 55017, 'MGI:1919081']
['fam160b1', 70126, 'MGI:2147545']
['4930452b06rik', 64726, 'MGI:1926154']


In [10]:
print (ls_notResponse)

['Dusp1', 'Cyr61', 'Rbm3', 'LOC100910163', 'Hist1h1c', 'LOC102551901', 'LOC500300', 'Ctgf', 'Cml3', 'RGD1560017', 'AABR07002120.1', 'LOC684871', 'LOC100912293', 'Dusp1', 'LOC100909555', 'LOC100363268', 'LOC500300', 'LOC102557302', 'LOC500594', 'LOC297756', 'Ctgf', 'RGD1561282', 'LOC100911993', 'Clec2j', 'Lyst', 'Igkc', 'LOC100360573', 'Rbm3', 'RGD1306926', 'Ccrn4l', 'RGD1564688', 'LOC499229', 'AABR07046778.1', 'LOC102552378', 'Paqr9', 'Mkl1', 'Elmsan1', 'Hist1h1d', 'AABR07015081.2', 'AABR07059879.1', 'LOC100363268', 'Lyst', 'LOC100360977', 'Rbm3', 'AABR07061336.3', 'Pak7', 'AABR07028989.1', 'LOC100366140', 'Ogn', 'LOC100910934', 'Cml5', 'Rbm3', 'LOC100360573', 'Tmem151b', 'Cecr6', 'Cml5', 'Dusp1', 'AABR07026359.1', 'Pqlc1', 'Lyst', 'Sdpr', 'Gramd3', 'Ddit4l2', 'Bcl7a', 'LOC100364791', 'Sh3bp5', 'Gucy1b3', 'Slc25a3', 'Rhoq', 'AABR07001905.1', 'RGD1562037', 'LOC100911874', 'LOC100912399', 'Cyr61', 'Pappa', 'Rn60_1_2787.1', 'H3f3b', 'LOC304239', 'Pragmin', 'LOC100912399', 'LOC100360638', 

In [11]:
# SECOND DICTIONARY

for i in range(len(mgi_id)):
    mgi_ID = mgi_id[i]
    gene_name = gene_name_v2[i]
    print(mgi_ID)
    
    if gene_name != '':

        # variables
        alias = ''

        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []
        

        ls_biomartParameters_mgi = biomartParameters_mgi(mgi_ID)
        
        print(ls_biomartParameters)
        
        if not ls_biomartParameters:
            ls_notResponse_after.append(mgi_ID)


        for ls in ls_biomartParameters:
            for j in range(3):
                if len(ls) < (j+1):
                    ls.append('')
            ensembl_gene_id_temp.append(ls[0])
            ensembl_transcript_id_temp.append(ls[1])
            refseq_mrna_temp.append(ls[2])  

        ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))
         
        
        ls_biomartHumanOrthologs = biomartHumanOrthologs_mgi(mgi_ID)
        if not ls_biomartHumanOrthologs:
            hgnc_symbol = ''
        else:
            hgnc_symbol = ls_biomartHumanOrthologs[0][0]       
        
        
        for k in range(len(ls_geneDictionaries)):
            temp = str(ls_geneDictionaries[k]['Gene_name']).lower()

            if temp == gene_name:
                print('response')
                ls_geneDictionaries[k]['Ensembl_gene_id'] = ensembl_gene_id
                ls_geneDictionaries[k]['Ensembl_transcript_id'] = ensembl_transcript_id
                ls_geneDictionaries[k]['RefSeq_mRNA_id'] = refseq_mrna
                ls_geneDictionaries[k]['HGNC_symbol'] = hgnc_symbol
                


MGI:88613
[]
response
response
MGI:1931526
[]
response
MGI:95537
[]
response
response
MGI:2136449
[]
response
MGI:95537
[]
response
response
MGI:109382
[]
response
response
MGI:2384495
[]
response
response
response
MGI:2685106
[]
response
response
MGI:107502
[]
response
MGI:1920334
[]
response
response
response
MGI:1916299
[]
response
response
response
MGI:2136977
[]
response
MGI:1916299
[]
response
response
response
MGI:1914193
[]
response
response
response
MGI:99513
[]
response
MGI:1914815
[]
response
response
MGI:1860604
[]
response
response
MGI:88613
[]
response
response
MGI:2442910
[]
response
MGI:1917820
[]
response
response
MGI:1923714
[]
response
response
MGI:1352760
[]
response
MGI:2384495
[]
response
response
response
MGI:2144224
[]
response
MGI:1915248
[]
response
MGI:2685304
[]
response
MGI:1917278
[]
response
response
response
MGI:97822
[]
response
MGI:1277968
[]
response
MGI:1917278
[]
response
response
response
MGI:1914815
[]
response
response
MGI:1920048
[]
response
MGI

In [12]:
# SCORES 

### from SECOND DICTIONARY to file

data3 = ls_geneDictionaries
df3 = pd.DataFrame.from_dict(data3)
df3.to_excel(r'.\secondDictionary.xlsx', index=False)
df3.to_csv(r'.\secondDictionary.tsv', sep="\t")

data4 = ls_notResponse_after
df4 = pd.DataFrame.from_dict(data4)
df4.to_excel(r'.\ls_notResponse_after.xlsx', index=False)



In [13]:
# Add ALIAS

mgi_file_path = r'..\MGI_EntrezGene.xlsx'
dictionary_file_path = r'.\secondDictionary.xlsx'
save_file_path = r'.\withAlias.xlsx'


updateCellswithAlias(mgi_file_path, dictionary_file_path, save_file_path)


In [14]:
# Add INFO

def updateCellswithINFO(source_file_path, source_sheet, alias_file_path, info_file_path):
    
    wb_info = openpyxl.load_workbook(source_file_path)
    ws_info = wb_info[source_sheet]
    ls_row_g_name = [ws_info.cell(row=i, column=3).value for i in range(2, ws_info.max_row + 1)]


    wb_alias = openpyxl.load_workbook(alias_file_path)
    ws_alias = wb_alias.active
    ls_row_2 = [ws_alias.cell(row=i, column=2).value for i in range(2, ws_alias.max_row + 1)]


    matched_indices = set()               # to keep track of matched id

    for j, (g_name, row_2) in enumerate(zip(ls_row_g_name, ls_row_2), start=2):
        if j not in matched_indices and str(g_name).lower() == str(row_2).lower():
            print(str(g_name).lower(), str(row_2).lower())
            L_cel = 'L' + str(j)
            tempInfo = str(ws_alias[L_cel].value)
            matched_indices.add(j)        # add matched index to set - avoid duplicate

### the number and type of items in the INFO column depends on the source publication


            ws_alias[L_cel] = ''
            ws_alias[L_cel] = ('Regulation:' + str(tempInfo).upper() + '|'
                            + 'Log2Ratio:' + str(ws_info['E' + str(j)].value) + '|'
                            + 'tissue:' + str(ws_info['H' + str(j)].value) + '|'
                            + 'cell:' + str(ws_info['I' + str(j)].value) + '|'
                            + 'enviroroment:' + str(ws_info['J' + str(j)].value) + '|'
                            + 'treatment:' + str(ws_info['K' + str(j)].value) + '|'
                            + 'dose:' + str(ws_info['L' + str(j)].value) + '|'
                            + 'Timepoint:' + str(ws_info['M' + str(j)].value) + '|'
                            + 'FDR_threshold:' + str(ws_info['N' + str(j)].value) + '|'
                            + 'method:' + str(ws_info['O' + str(j)].value) + '|'
                            + 'gene_element:' + str(ws_info['P' + str(j)].value) + '|'
                            + 'FDR:' + str(ws_info['Q' + str(j)].value) + '|'
                            + 'statistics:' + str(ws_info['R' + str(j)].value))

    # Save the file
    wb_alias.save(info_file_path)



source_file_path = r'..\ifpan-GR-database-papers.xlsx'
source_sheet = 'hipocamp-fs'
alias_file_path = '.\withAlias.xlsx'
info_file_path = r'.\withINFO.xlsx'

updateCellswithINFO(source_file_path, source_sheet, alias_file_path, info_file_path)


apold1 apold1
dusp1 dusp1
cyr61 cyr61
btg2 btg2
fos fos
nr4a1 nr4a1
sgk1 sgk1
nfkbia nfkbia
junb junb
zfp189 zfp189
atf3 atf3
sik1 sik1
egr1 egr1
zfp36 zfp36
arc arc
zfp180 zfp180
klf2 klf2
arrdc3 arrdc3
dbp dbp
fosb fosb
per1 per1
rbm3 rbm3
loc100910163 loc100910163
aabr07001512.1 aabr07001512.1
aabr07001910.1 aabr07001910.1
plac9 plac9
dll4 dll4
hist1h1c hist1h1c
ifit3 ifit3
klf4 klf4
loc102551901 loc102551901
errfi1 errfi1
loc500300 loc500300
bhlhe40 bhlhe40
ac107580.1 ac107580.1
dusp6 dusp6
ctgf ctgf
tmem252 tmem252
npas4 npas4
egr2 egr2
cml3 cml3
ak3 ak3
oca2 oca2
rgd1560017 rgd1560017
tmem119 tmem119
tnnt1 tnnt1
aabr07002120.1 aabr07002120.1
loc684871 loc684871
rgs1 rgs1
arl4a arl4a
bloc1s2 bloc1s2
gpd1 gpd1
ppp1r15a ppp1r15a
pomp pomp
loc100912293 loc100912293
sgk1 sgk1
nfkbia nfkbia
nr4a1 nr4a1
arc arc
apold1 apold1
errfi1 errfi1
fos fos
per1 per1
dusp1 dusp1
zfp189 zfp189
junb junb
sik1 sik1
gpd1 gpd1
pdk4 pdk4
plekhf1 plekhf1
egr1 egr1
slc2a1 slc2a1
tmem252 tmem252
egr2 egr2


nedd4l nedd4l
ube2e2 ube2e2
cntn6 cntn6
slit1 slit1
pinx1 pinx1
xkr4 xkr4
hbb-b1 hbb-b1
adgrb2 adgrb2
ccdc186 ccdc186
clec18a clec18a
enkur enkur
fah fah
hip1 hip1
ldlrad4 ldlrad4
mical2 mical2
tmtc3 tmtc3
rnf10 rnf10
svil svil
suv39h2 suv39h2
aabr07054405.1 aabr07054405.1
tmem229a tmem229a
prickle2 prickle2
abhd3 abhd3
gse1 gse1
lrriq3 lrriq3
msr1 msr1
canx canx
cr1l cr1l
gtf2ird1 gtf2ird1
pam pam
tmtc2 tmtc2
fdps fdps
gsdmd gsdmd
nxn nxn
arntl arntl
per3 per3
dbp dbp
per2 per2
oas2 oas2
adamts9 adamts9
spock1 spock1
ntng1 ntng1
rpl19 rpl19
nr1d2 nr1d2
tef tef
anxa9 anxa9
unc5d unc5d
pld5 pld5
parvb parvb
npas2 npas2
cog2 cog2
slc25a31 slc25a31
cbs cbs
adamts16 adamts16
spata17 spata17
pdzd2 pdzd2
cd163 cd163
kcnh7 kcnh7
rgs6 rgs6
zfp536 zfp536
entpd5 entpd5
egfr egfr
mocos mocos
zfat zfat
ptchd4 ptchd4
myo7b myo7b
psap psap
sft2d2 sft2d2
hmgcs1 hmgcs1
s100a3 s100a3
wipf3 wipf3
egfl6 egfl6
eepd1 eepd1
anln anln
f13a1 f13a1
slc27a5 slc27a5
ndst4 ndst4
rgd1560034 rgd1560034
sult5a1 sult

In [ ]:
                                ''' + '|'
                                 + '|'
                                +'TTEST.NEURO.ALD:' + str(ws_mgi[S_cel].value) + '|'
                                +'TTEST.NEURO.DHT:' + str(ws_mgi[T_cel].value) + '|'
                                +'TTEST.NEURO.PRG:' + str(ws_mgi[U_cel].value) + '|'
                                +'TTEST.NEURO.ESR:' + str(ws_mgi[V_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.DEX:' + str(ws_mgi[W_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.ALD:' + str(ws_mgi[X_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.DHT:' + str(ws_mgi[Y_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.PRG:' + str(ws_mgi[Z_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.ESR:' + str(ws_mgi[AA_cel].value) + '|'
                                +'FDR_TTEST.NEURO.DEX:' + str(ws_mgi[AB_cel].value) + '|'
                                +'FDR_TTEST.NEURO.ALD:' + str(ws_mgi[AC_cel].value) + '|'
                                +'FDR_TTEST.NEURO.DHT:' + str(ws_mgi[AD_cel].value) + '|'
                                +'FDR_TTEST.NEURO.PRG:' + str(ws_mgi[AE_cel].value) + '|'
                                +'FDR_TTEST.NEURO.ESR:' + str(ws_mgi[AF_cel].value) + '|'
                                +'FOLD.ASTRO.DEX:' + str(ws_mgi[AG_cel].value) + '|'
                                +'FOLD.ASTRO.ALD:' + str(ws_mgi[AH_cel].value) + '|'
                                +'FOLD.ASTRO.DHT:' + str(ws_mgi[AI_cel].value) + '|'
                                +'FOLD.ASTRO.PRG:' + str(ws_mgi[AJ_cel].value) + '|'
                                +'FOLD.ASTRO.ESR:' + str(ws_mgi[AK_cel].value) + '|'
                                +'FOLD.NEURO.DEX:' + str(ws_mgi[AL_cel].value) + '|'
                                +'FOLD.NEURO.ALD:' + str(ws_mgi[AM_cel].value) + '|'
                                +'FOLD.NEURO.DHT:' + str(ws_mgi[AN_cel].value) + '|'
                                +'FOLD.NEURO.PRG:' + str(ws_mgi[AO_cel].value) + '|'
                                +'FOLD.NEURO.ESR:' + str(ws_mgi[AP_cel].value))'''